### 직방 원룸 매물 데이터 수집

- 절차
    - 동 이름 > 위도, 경도
    - 위도, 경도 > geohash 변환
        - geohash: 위도, 경도 값 -> 영역 값 변환
    - geohash > 매물 아이디
    - 매물 아이디 > 매물 정보

In [1]:
import requests
import pandas as pd
import geohash2

In [5]:
# 1. 동이름으로 위도 경도 구하기

addr = "강남구 역삼동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.495365142822266, 127.03306579589844)

In [6]:
# 2. 위도 경도로 geohash 알아내기

geohash = geohash2.encode(lat, lng, precision=5) # precision이 커질수록 영역이 작아짐
geohash

'wydm6'

In [7]:
# 3. geohash로 매물 아이디 가져오기

url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
response = requests.get(url)
datas = response.json()["items"]
# len(datas), datas[0]
ids = [data["item_id"] for data in datas]
len(ids), ids[:5] # 1000개 넘어가면 나눠서 수집해야 함

(2346, [32819635, 32838798, 32872608, 32838700, 32767020])

In [9]:
# 4. 매물 아이디로 매물 정보 가져오기

url = "https://apis.zigbang.com/v2/items/list"
params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": ids[:900]
}
response = requests.post(url, params) # params 내에 한글이 없으므로 json.dumps()로 인코딩 할 필요 X
response

<Response [200]>

In [10]:
datas = response.json()["items"]
df = pd.DataFrame(datas)
df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,...,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
898,None,32809154,https://ic.zigbang.com/ic/items/32809154/1.jpg,월세,월세,120,70,19.83,"{'m2': 19.83, 'p': '6'}","{'m2': 19.83, 'p': '6'}",...,False,True,원룸,[추천],서울시 강남구 역삼동,None,None,9,2022-07-29T15:01:31+09:00,False
899,None,32844154,https://ic.zigbang.com/ic/items/32844154/1.jpg,월세,월세,100,100,29.75,"{'m2': 29.75, 'p': '9'}","{'m2': 29.75, 'p': '9'}",...,False,True,원룸,[추천],서울시 강남구 역삼동,None,None,9,2022-08-01T18:43:35+09:00,False


In [11]:
# 필요한 컬럼만 필터링
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "floor", "building_floor",
           "address1", "manage_cost"]
filtered_column_df = df[columns]
filtered_column_df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost
898,32809154,월세,120,70,19.83,3,4,서울시 강남구 역삼동,9
899,32844154,월세,100,100,29.75,2,2,서울시 강남구 역삼동,9


In [12]:
# 주소에 검색할 동이 있는 데이터만 필터링 -> geohash로 범위값이 설정되어 다른 동도 포함됨
result_df = filtered_column_df[filtered_column_df["address1"].str.contains(addr)]
result_df = result_df.reset_index(drop=True)
result_df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,address1,manage_cost
563,32809154,월세,120,70,19.83,3,4,서울시 강남구 역삼동,9
564,32844154,월세,100,100,29.75,2,2,서울시 강남구 역삼동,9


In [14]:
# 함수 만들기
def oneroom(addr):
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["item_id"] for data in datas]

    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": ids[:900]
    }
    response = requests.post(url, params)
    
    datas = response.json()["items"]
    df = pd.DataFrame(datas)
    
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    df_filtered = df[columns]
    result_df = df_filtered[df_filtered["address1"].str.contains(addr)]
    result_df = result_df.reset_index(drop=True)
    display(result_df)
    

In [16]:
oneroom('진천동')

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32823096,월세,1000,30,29.75,대구시 달서구 진천동,5
1,32858432,월세,200,25,29.75,대구시 달서구 진천동,10
2,32576871,월세,500,30,30.41,대구시 달서구 진천동,10
3,32829890,월세,300,33,26.45,대구시 달서구 진천동,5
4,32852556,월세,300,28,25.00,대구시 달서구 진천동,10
...,...,...,...,...,...,...,...
100,32661356,월세,300,25,29.75,대구시 달서구 진천동,10
101,32821209,월세,300,25,23.14,대구시 달서구 진천동,10
102,32758589,월세,300,22,29.75,대구시 달서구 진천동,10
103,32875895,월세,3000,45,49.59,대구시 달서구 진천동,10


In [ ]:
# 모듈 파일 만들기(.py)
# magic command (%%writefile)은 cell의 맨 첫 줄에 있어야함

In [19]:
%%writefile zigbang.py

import requests, geohash2
import pandas as pd

def oneroom(addr):
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"
    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    geohash = geohash2.encode(lat, lng, precision=5)
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세\
&service_type_eq=원룸"
    response = requests.get(url)
    datas = response.json()["items"]
    ids = [data["item_id"] for data in datas]

    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": ids[:900]
    }
    response = requests.post(url, params)
    
    datas = response.json()["items"]
    df = pd.DataFrame(datas)
    
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    df_filtered = df[columns]
    result_df = df_filtered[df_filtered["address1"].str.contains(addr)]
    result_df = result_df.reset_index(drop=True)
    
    return result_df 

Writing zigbang.py


In [20]:
import zigbang as zb
df = zb.oneroom("대명동")
df.head()

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32548317,월세,500,30,23.14,대구시 남구 대명동,10
1,32748249,월세,300,28,26.45,대구시 남구 대명동,10
2,32621371,월세,500,35,33.06,대구시 남구 대명동,10
3,32411326,월세,3000,70,69.00,대구시 남구 대명동,10
4,32579885,월세,300,30,26.45,대구시 남구 대명동,5


In [ ]:
%whos 
# 변수 목록

In [ ]:
%reset 
# 변수 초기화